<a href="https://colab.research.google.com/github/genterist/BKT-Jupyter/blob/master/BKT_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://brand.ncsu.edu/img/logo/brick2x2.jpg =170x)

## **DEPARTMENT OF COMPUTER SCIENCE - CSC591 FALL 2018 Edu Tech**
	
###TAM N. NGUYEN

In [0]:
# PLEASE RUN THIS FIRST

import sys
version = sys.version_info[0], sys.version_info[1]
print("Running on Python {}.{}".format(version[0], version[1]))

# Imports for all questions
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
#from sklearn import preprocessing, svm
#from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import LeaveOneOut
#from sklearn import metrics
#from math import sqrt
#from keras.models import Sequential
#from keras.layers import Dense
#from itertools import combinations
#
# ATTENTION
#
# Please read README file to run codes properly in Jupyter Notebook
#
# Running on Google Colab notebook will require manual upload of CSV files to the "Files" section
# in the left column of the interface. 


Running on Python 2.7


### DATA IMPORT and PRE-PROCESSING

In [0]:
# for Google Colab, make sure you upload data at the beginning of EACH new session
# make sure you run code block 1 of this jupyter notebook (for imports)
#
raw_data = pd.read_csv('./data.csv', sep=',')
raw_data.shape

(36627, 10)

In [0]:
# original structure
raw_data.head(3)

,Student,StepID,Correct,KC_1,KC_27,KC_24,KC_14,KC_22,KC_20,KC_21
0,stu1,478,0,0,1,0,0,0,0,0
1,stu1,479,1,0,1,0,0,0,0,0
2,stu1,484,1,0,1,0,0,0,0,0


In [0]:
# turning it to a multi-index DF
mod1_data = raw_data.set_index(['Student','StepID'])

#make sure everything is in order
mod1_data.sortlevel(inplace=True)

# insert collumns for the probabilities
# hard-coding of columns is fine here since this is a simulated scenario
# real life situations may involve json or
# regular db which is much more organized

mod1_data.insert(2,'P(L)1',0.0)
mod1_data.insert(3,'P(C)1',0.0)

mod1_data.insert(5,'P(L)27',0.0)
mod1_data.insert(6,'P(C)27',0.0)

mod1_data.insert(8,'P(L)24',0.0)
mod1_data.insert(9,'P(C)24',0.0)

mod1_data.insert(11,'P(L)14',0.0)
mod1_data.insert(12,'P(C)14',0.0)

mod1_data.insert(14,'P(L)22',0.0)
mod1_data.insert(15,'P(C)22',0.0)

mod1_data.insert(17,'P(L)20',0.0)
mod1_data.insert(18,'P(C)20',0.0)

mod1_data.insert(20,'P(L)21',0.0)
mod1_data.insert(21,'P(C)21',0.0)

# only P(C) matter since it has other Probabilities embedded in it

mod1_data.head(5)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  after removing the cwd from sys.path.


Correct  KC_1  P(L)1  P(C)1  KC_27  P(L)27  P(C)27  KC_24  \
Student StepID                                                              
stu1    478           0     0    0.0    0.0      1     0.0     0.0      0   
        479           1     0    0.0    0.0      1     0.0     0.0      0   
        484           1     0    0.0    0.0      1     0.0     0.0      0   
        485           1     0    0.0    0.0      1     0.0     0.0      0   
        497           1     0    0.0    0.0      0     0.0     0.0      0   

                P(L)24  P(C)24   ...    P(C)14  KC_22  P(L)22  P(C)22  KC_20  \
Student StepID                   ...                                           
stu1    478        0.0     0.0   ...       0.0      0     0.0     0.0      0   
        479        0.0     0.0   ...       0.0      0     0.0     0.0      0   
        484        0.0     0.0   ...       0.0      0     0.0     0.0      0   
        485        0.0     0.0   ...       0.0      0     0.0     0.0      0   
        497        0.0     0.0   ...       0.0      1     0.0     0.0      0   

                P(L)20  P(C)20  KC_21  P(L)21  P(C)21  
Student StepID                                         
stu1    478        0.0     0.0      0     0.0     0.0  
        479        0.0     0.0      0     0.0     0.0  
        484        0.0     0.0      0     0.0     0.0  
        485        0.0     0.0      0     0.0     0.0  
        497        0.0     0.0      0     0.0     0.0  

[5 rows x 22 columns]

### PROBABILITY FUNCTIONS

![alt text](https://i.imgur.com/DbDMC5d.png)

Credit: wikipedia

u : student \\
k : skill \\
function d: skill mastery \\
function e: correct application of skill in the future

We assume these following parameters are fixed:

P(Lo) = 0.5

P(T) = 0.1

P(S) = 0.1

P(G) = 0.1

per

Hawkins W.J., Heffernan N.T., Baker R.S.J.D. (2014) **Learning Bayesian Knowledge Tracing Parameters with a Knowledge Heuristic and Empirical Probabilities.** In: Trausan-Matu S., Boyer K.E., Crosby M., Panourgia K. (eds) Intelligent Tutoring Systems. ITS 2014. Lecture Notes in Computer Science, vol 8474. Springer, Cham

Previous efforts regarding the science behind the setting of these parameters can be found in papers such as:

Yudelson M.V., Koedinger K.R., Gordon G.J. (2013) I**ndividualized Bayesian Knowledge Tracing Models**. In: Lane H.C., Yacef K., Mostow J., Pavlik P. (eds) Artificial Intelligence in Education. AIED 2013. Lecture Notes in Computer Science, vol 7926. Springer, Berlin, Heidelberg

We can also use machine learning approaches to change these parameters along the way.



In [0]:
P_L0 = 0.5
P_T = 0.1
P_S = 0.1
P_G = 0.1

#function definition here

def P_L_func ( correct, P_L_previous):
  
  if correct==1:
    P_L_obs = (P_L_previous*(1-P_S))/(P_L_previous*(1-P_S) + (1-P_L_previous)*(1-P_G))
  else:
    P_L_obs = (P_L_previous*P_S)/(P_L_previous*P_S + (1-P_L_previous)*(1-P_G))
  
  P_L_current = P_L_obs + (1-P_L_obs)*P_T
  
  return P_L_current

def P_C_func (P_L_previous):
  P_C_current = P_L_previous*(1-P_S) + (1-P_L_previous)*P_G
  
  return P_C_current

### FILLING IN CALCULATED RESULTS

LOREM IPSUM

In [0]:
for Student, stuInfo in mod1_data.groupby(level=[0]):
  rows = len(stuInfo.index)
  row_loc = 0

  print('Student ID: %s' %(Student))
  print('Logged actions: %s' %(rows))
  
  # set the innitial P
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)1')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)1')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)27')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)27')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)24')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)24')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)14')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)14')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)22')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)22')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)20')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)20')] = P_L0

  stuInfo.iloc[0,stuInfo.columns.get_loc('P(L)21')] = P_L0
  stuInfo.iloc[0,stuInfo.columns.get_loc('P(C)21')] = P_L0
  
  # now filling in values in a per-column fashion
  for index, row in stuInfo.iterrows():
    
    # the first row was innitialized so we skip it now
    if row_loc >0:
    
      #going over KC_1	
      if stuInfo.iloc[row_loc]['KC_1'] == 0.0:
        #if the skill is not tested then P(L) and P(C) are unchanged and will be copied over
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)1')] = stuInfo.iloc[row_loc-1]['P(L)1']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)1')] = stuInfo.iloc[row_loc-1]['P(C)1']
      else:
        #if the skill is tested, change P(L) and P(C) accordingly
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)1')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)1'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)1')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)1'])
    
    
      #going over KC_27
      if stuInfo.iloc[row_loc]['KC_27'] == 0.0:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)27')] = stuInfo.iloc[row_loc-1]['P(L)27']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)27')] = stuInfo.iloc[row_loc-1]['P(C)27']
      else:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)27')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)27'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)27')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)27'])      
    
    
      #going over KC_24	
      if stuInfo.iloc[row_loc]['KC_24'] == 0.0:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)24')] = stuInfo.iloc[row_loc-1]['P(L)24']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)24')] = stuInfo.iloc[row_loc-1]['P(C)24']
      else:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)24')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)24'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)24')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)24'])
        
      #going over KC_14	
      if stuInfo.iloc[row_loc]['KC_14'] == 0.0:
        #if the skill is not tested then P(L) and P(C) are unchanged and will be copied over
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)14')] = stuInfo.iloc[row_loc-1]['P(L)14']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)14')] = stuInfo.iloc[row_loc-1]['P(C)14']
      else:
        #if the skill is tested, change P(L) and P(C) accordingly
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)14')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)14'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)14')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)14'])
        
      #going over KC_22	
      if stuInfo.iloc[row_loc]['KC_22'] == 0.0:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)22')] = stuInfo.iloc[row_loc-1]['P(L)22']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)22')] = stuInfo.iloc[row_loc-1]['P(C)22']
      else:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)22')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)22'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)22')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)22'])
        
      #going over KC_20	
      if stuInfo.iloc[row_loc]['KC_20'] == 0.0:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)20')] = stuInfo.iloc[row_loc-1]['P(L)20']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)20')] = stuInfo.iloc[row_loc-1]['P(C)20']
      else:
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)20')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)20'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)20')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)20'])
        
      #going over KC_21
      if stuInfo.iloc[row_loc]['KC_21'] == 0.0:
        #if the skill is not tested then P(L) and P(C) are unchanged and will be copied over
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)21')] = stuInfo.iloc[row_loc-1]['P(L)21']
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)21')] = stuInfo.iloc[row_loc-1]['P(C)21']
      else:
        #if the skill is tested, change P(L) and P(C) accordingly
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(L)21')] = P_L_func ( stuInfo.iloc[row_loc]['Correct'], stuInfo.iloc[row_loc-1]['P(L)21'])
        stuInfo.iloc[row_loc,stuInfo.columns.get_loc('P(C)21')] = P_C_func ( stuInfo.iloc[row_loc-1]['P(L)21'])
        
    if row_loc < rows: row_loc += 1
    #print row_loc #this is for step by step visual check
  
  # for checking results of each col
  # make sure "break" in the below line is enabled
  #print(stuInfo['P(L)1'],stuInfo['P(C)1'])
  
  print(stuInfo.tail(1))
  
  # break here for trouble shooting the first student only
  #break
#mod1_data.head(5)

Student ID: stu1
Logged actions: 311


/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


                Correct  KC_1     P(L)1    P(C)1  KC_27    P(L)27    P(C)27  \
Student StepID                                                                
stu1    5064          1     0  0.704755  0.63756      0  0.609475  0.552867   

                KC_24    P(L)24    P(C)24    ...       P(C)14  KC_22  \
Student StepID                               ...                       
stu1    5064        0  0.675196  0.611286    ...     0.438831      0   

                  P(L)22    P(C)22  KC_20    P(L)20    P(C)20  KC_21  \
Student StepID                                                         
stu1    5064    0.477479  0.435537      1  0.437557  0.400051      0   

                  P(L)21    P(C)21  
Student StepID                      
stu1    5064    0.901954  0.812848  

[1 rows x 22 columns]
Student ID: stu10
Logged actions: 426
                Correct  KC_1     P(L)1     P(C)1  KC_27    P(L)27    P(C)27  \
Student StepID                                                              

In [0]:
# export data frame to local .csv file
from google.colab import files
mod1_data.to_csv('results.csv', index=False)
files.download('results.csv')